In [ ]:
from langchain_groq import ChatGroq
import uuid
from dotenv import load_dotenv
import os

load_dotenv()

: 

In [19]:
llm = ChatGroq(
    groq_api_key = os.getenv('GROQ_API_KEY'),
    model = "llama-3.1-70b-versatile",
    temperature = 0 ,
)

In [20]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader( 'https://jobs.michelin.in/job-offer-result-list/software-engineer-python')
page_data = loader.load().pop().page_content
print(page_data)

















  Software Engineer - Python
  









Logo















Why Michelin



Return


Inspire Motion for Life


Your professional journey





About us



Return


What inspires us?


Michelin in the world


Students


Meet our recruiters




Michelin in India











Software Engineer - Python

Offer published on 27 08 2024

Location :
Pune, India
            
Sector :
IS&Digital
            
Contract type :
Regular
            

Apply




Open positions

Software Engineer - Python- - - - - - - - - - - -Key Expected Skills Python languagePython libraryORMMulti process architectureVersion control tool as GitExperience on building from scratch Python applicationSQLOptional skillsDjango FrameworkExperience on front end technology (Angular)SQL (Postgre or Oracle)Docker and KubernetesJava back end SpringData experienceAzure CloudMicro servicesAPI DevSoft Skills MandatoryGood English communication (worldwide user and part of the team in France) Good level of autonomy (sma

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### Scapped from website :
    {page_data}
    ### Instruction :
    The Scrapped data is from the career's page of a website.
    Your job is to extract the job postings and return them in  JSON format containing the followings keys :
    'role' , 'experience' , 'skills' and description'.
    Only return th e valid JSON.
    ### valid JSON (NO PREAMBLE And AVOID Optional Skills) :
    """
)

chain_extract = prompt_extract | llm

res = chain_extract.invoke(input = {'page_data' : page_data})

print(res.content)

```json
{
  "role": "Software Engineer - Python",
  "experience": "",
  "skills": [
    "Python language",
    "Python library",
    "ORM",
    "Multi process architecture",
    "Version control tool as Git",
    "Experience on building from scratch Python application",
    "SQL"
  ],
  "description": "Good English communication (worldwide user and part of the team in France) Good level of autonomy (small team (2 or 3) => Lot of responsibility for all team member."
}
```


In [27]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

In [28]:
type(json_res) 

dict

## ChromaDb portfolio add

In [29]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [30]:
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _ , row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links" : row["Links"]},
                       ids = [str(uuid.uuid4())] )

## querying DB to extract relevent portfolio

In [31]:
job = json_res
job['skills']

['Python language',
 'Python library',
 'ORM',
 'Multi process architecture',
 'Version control tool as Git',
 'Experience on building from scratch Python application',
 'SQL']

In [32]:
links = collection.query(
    query_texts = job['skills'],
    n_results =2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

## Cold Email Generator (+prompt)

In [33]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION :
    {job_description}

    ### INSTRUCTION :
    You are Animesh , a business development executive at some service providing company .
    Your company(say any name) is an AI and Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of company 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase company's portfolio: {link_list}
        Remember you are Mohan, BDE at company. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE with any artificial company name):
        
        """
)

chain_email = prompt_email | llm 
res = chain_email.invoke({'job_description' : str(job)  , 'link_list' : links})
print(res.content)

Subject: Expert Python Development Services for Your Business Needs

Dear Hiring Manager,

I came across your job posting for a Software Engineer - Python, and I'm excited to introduce you to TechForge, an AI and Software Consulting company dedicated to delivering tailored solutions that drive business growth through automation.

Our team of expert Python developers has extensive experience in building scalable, efficient, and secure applications from scratch. We're well-versed in Python language, libraries, ORM, multi-process architecture, and version control tools like Git. Our expertise in SQL and experience with building Python applications align perfectly with your requirements.

At TechForge, we've empowered numerous enterprises with our solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* https://example.com/ml-python-portfo